In [1]:
import scanpy as sc
import scanpy.external as sce
import tempfile
import anndata as ad
import os
import torch
import scvi
import ray
import matplotlib.pyplot as plt
from ray import tune
from scvi import autotune
torch.set_float32_matmul_precision('high')
os.chdir("/data")


In [2]:
scvi.__version__

'1.2.0'

In [3]:
torch.cuda.is_available()

True

In [4]:
adata = sc.read_h5ad("h5ad/doublet_filtered.h5ad")

In [5]:
scvi.data.poisson_gene_selection(adata, layer='counts', n_top_genes=8000, subset=True, batch_key="sample_id") # selecting HVGs using poisson sampling

Sampling from binomial...:   0%|          | 0/10000 [00:00<?, ?it/s]

Sampling from binomial...:   0%|          | 0/10000 [00:00<?, ?it/s]

Sampling from binomial...:   0%|          | 0/10000 [00:00<?, ?it/s]

Sampling from binomial...:   0%|          | 0/10000 [00:00<?, ?it/s]

Sampling from binomial...:   0%|          | 0/10000 [00:00<?, ?it/s]

Sampling from binomial...:   0%|          | 0/10000 [00:00<?, ?it/s]

Sampling from binomial...:   0%|          | 0/10000 [00:00<?, ?it/s]

Sampling from binomial...:   0%|          | 0/10000 [00:00<?, ?it/s]

In [7]:
model_cls = scvi.model.SCVI

model_cls.setup_anndata(adata, layer="counts", batch_key='sample_id')

search_space = {
    "model_params": {"n_hidden": tune.choice([64, 128, 256]), 
                     "n_layers": tune.choice([1, 2, 3, 4]),
                     "n_latent": tune.choice([10, 20, 30, 40, 50]),
                     "gene_likelihood": tune.choice(["nb", "zinb"])
                    },
    "train_params": {"max_epochs": 100,
                     "plan_kwargs": {"lr": tune.loguniform(1e-4, 1e-2)}}}

In [8]:
ray.init(log_to_driver=False)

/home/ubuntu/miniconda3/envs/scanpy/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-10-08 20:07:03,552	INFO worker.py:1724 -- Started a local Ray instance.


Python version:,3.10.13
Ray version:,2.9.3


In [ ]:
results = autotune.run_autotune(
    model_cls,
    data=adata,
    mode="min",
    metrics="validation_loss",
    search_space=search_space,
    num_samples=120,
    resources={"cpu": 5, "gpu": 1},
    logging_dir="/opt/dlami/nvme"    
)

In [ ]:
import pickle
with open("nb/scvi/autotune_results.pickle", "wb") as fp:
    pickle.dump(results, file=fp)

In [17]:
scvi_tuner.info()

ModelTuner registry for SCVI

                  Tunable hyperparameters                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃      Hyperparameter      ┃ Default value ┃    Source    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│         n_hidden         │      128      │     VAE      │
│         n_latent         │      10       │     VAE      │
│         n_layers         │       1       │     VAE      │
│       dropout_rate       │      0.1      │     VAE      │
│        dispersion        │     gene      │     VAE      │
│     log_variational      │     True      │     VAE      │
│     gene_likelihood      │     zinb      │     VAE      │
│   latent_distribution    │    normal     │     VAE      │
│    encode_covariates     │     False     │     VAE      │
│ deeply_inject_covariates │     True      │     VAE      │
│      use_batch_norm      │     both      │     VAE      │
│      use_layer_norm      │     none      │     VAE      │
│  use_observed_lib_size   │     True      │     VAE      │
│      var_activation      │     None      │     VAE      │
│        optimizer         │     Adam      │ TrainingPlan │
│            lr            │     0.001     │ TrainingPlan │
│       weight_decay       │     1e-06     │ TrainingPlan │
│           eps            │     0.01      │ TrainingPlan │
│    n_steps_kl_warmup     │     None      │ TrainingPlan │
│    n_epochs_kl_warmup    │      400      │ TrainingPlan │
│   reduce_lr_on_plateau   │     False     │ TrainingPlan │
│        lr_factor         │      0.6      │ TrainingPlan │
│       lr_patience        │      30       │ TrainingPlan │
│       lr_threshold       │      0.0      │ TrainingPlan │
│          lr_min          │       0       │ TrainingPlan │
│      max_kl_weight       │      1.0      │ TrainingPlan │
│      min_kl_weight       │      0.0      │ TrainingPlan │
│        batch_size        │      128      │     SCVI     │
└──────────────────────────┴───────────────┴──────────────┘

       Available metrics        
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃     Metric      ┃    Mode    ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ validation_loss │    min     │
└─────────────────┴────────────┘

                         Default search space                         
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Hyperparameter ┃ Sample function ┃  Arguments  ┃ Keyword arguments ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│    n_hidden    │     choice      │ [[64, 128]] │        {}         │
└────────────────┴─────────────────┴─────────────┴───────────────────┘

In [1]:
search_space = {
    "n_hidden": tune.choice([64, 128, 256]),
    "n_layers": tune.choice([1, 2, 3, 4]),
    "n_latent": tune.choice([10, 20, 30, 50, 75]),
    "lr": tune.loguniform(1e-4, 1e-2),
    "gene_likelihood": tune.choice(["nb", "zinb"])
} #parameters to test

NameError: name 'tune' is not defined

In [19]:
ray.init(log_to_driver=False)

/home/ubuntu/miniconda3/envs/scanpy/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-07-15 20:10:19,407	INFO worker.py:1724 -- Started a local Ray instance.


Python version:,3.10.14
Ray version:,2.9.3


In [ ]:
results = scvi_tuner.fit(
    adata,
    metric="validation_loss",
    additional_metrics=["elbo_validation"],
    search_space=search_space,
    num_samples=120,
    max_epochs=100,
    resources={"cpu": 6, "gpu": 1},
)

2024-07-15 20:10:21,999	INFO tune.py:583 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


In [1]:
import pickle

In [ ]:
with open("nb/scvi/autotune_results.pickle", "wb") as fp:
    pickle.dump(results, file=fp)

In [ ]:
!sudo poweroff

In [2]:
with open("/home/ubuntu/TATI_SCRNA/live-dead/nb/scvi/autotune_results.pickle", "rb") as fp:
       x=pickle.load(fp)

In [14]:
x.results.get_dataframe().to_csv("hp_search.csv")

In [15]:
x

TuneAnalysis(model_kwargs={'n_hidden': 256, 'n_layers': 2, 'n_latent': 30, 'gene_likelihood': 'nb'}, train_kwargs={'plan_kwargs': {'lr': 0.00815519881021457}}, metric={'metric': 'validation_loss', 'mode': 'min', 'value': 5934.95263671875}, additional_metrics={}, search_space={'n_hidden': <ray.tune.search.sample.Categorical object at 0x7f628f3d2860>, 'n_layers': <ray.tune.search.sample.Categorical object at 0x7f628f3d3220>, 'n_latent': <ray.tune.search.sample.Categorical object at 0x7f6083473310>, 'lr': <ray.tune.search.sample.Float object at 0x7f607fc6b670>, 'gene_likelihood': <ray.tune.search.sample.Categorical object at 0x7f607fc6bca0>}, results=ResultGrid<[
  Result(
    metrics={'validation_loss': 6028.18310546875},
    path='/home/ubuntu/TATI_SCRNA/live-dead/nb/scvi/scvi_log/autotune/2024-07-15_20-10-20_scvi/_trainable_fe0fe369_1_gene_likelihood=nb,lr=0.0093,n_hidden=64,n_latent=30,n_layers=1_2024-07-15_20-10-22',
    filesystem='local',
    checkpoint=None
  ),
  Result(
    metr